use loc[i, field] instead of iloc[i, field], iloc is very slow.

In [1]:
import sys
sys.path = ['/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/RecStudio/'] + sys.path
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4"
import random
import numpy as np
import pandas as pd
# import cudf, itertools
import scipy.sparse as ssp
from functools import lru_cache, partial
from tqdm import tqdm, trange
from collections import Counter, defaultdict
import torch
import pickle


In [2]:
def cast_dtype(df : pd.DataFrame, columns=None):
    if columns is None:
        columns = df.columns
    for k in columns:
        dt = type(df[k].iloc[0])
        if 'float' in str(dt):
            df[k] = df[k].astype('float32')
        elif 'int' in str(dt):
            df[k] = df[k].astype('int32')
        elif dt == list:
            dt_ = type(df.iloc[0][k][0])
            if 'float' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.float32))
            elif 'int' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.int32))

In [3]:
def _load_cache(path):
    with open(path, 'rb') as f:
        download_obj = pickle.load(f)
    return download_obj

In [4]:
def get_scores(merged_candidates_df, query_embeddings, product_embeddings):
    batch_size = 1024
    num_iter = (len(merged_candidates_df) - 1) // batch_size + 1
    score_list = []
    with torch.no_grad():
        for i in tqdm(range(num_iter)):
            st, ed = i * batch_size, (i + 1) * batch_size 
            batch_sess = merged_candidates_df.iloc[st : ed]
            batch_sess_id = torch.tensor(batch_sess['sess_id'].tolist(), dtype=torch.long, device=query_embeddings.device)
            batch_product_id = torch.tensor(batch_sess['dataset_id'].tolist(), dtype=torch.long, device=product_embeddings.device)
            query_emb = query_embeddings[batch_sess_id]
            product_emb = product_embeddings[batch_product_id]
            batch_score = (query_emb * product_emb).sum(dim=-1) 
            score_list.append(batch_score.cpu())
        score_list = torch.cat(score_list, dim=0).cpu().tolist()
        return score_list 

In [5]:
def normalize_scores(score_df, score_name, normalized_score_name):
    # score_df_g = cudf.from_pandas(score_df)
    score_df['exp_score'] = np.exp(score_df[score_name].to_numpy())
    scores_sum = score_df[['sess_id', 'exp_score']].groupby('sess_id').sum()
    scores_sum.reset_index(inplace=True)
    scores_sum = scores_sum.sort_values(by=['sess_id'], ascending=True)
    scores_sum.reset_index(drop=True, inplace=True)
    scores_sum.rename(columns={'exp_score' : 'score_sum'}, inplace=True)

    merged_score_df = score_df.merge(scores_sum, how='left', left_on=['sess_id'], right_on=['sess_id'])
    merged_score_df = merged_score_df.sort_values(by=['sess_id', 'product'])
    merged_score_df.reset_index(drop=True, inplace=True)
    
    # merged_score_df = merged_score_df_g.to_pandas(merged_score_df_g)
    score_df[normalized_score_name] = merged_score_df['exp_score'] / merged_score_df['score_sum']
    score_df['exp_score'] = merged_score_df['exp_score']
    score_df['score_sum'] = merged_score_df['score_sum']

    # del scores_sum_g
    # del merged_score_df_g 

# Merge test score

In [6]:
FIELD_NAME = 'narm_feat_scores'

In [7]:
merged_candidates_feature_test_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates_phase2/merged_candidates_150_test_feature.parquet'
test_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/sessions_test_task1_phase2.csv'

In [24]:
DE_product_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates_phase2/query_embeddings/NARM_Feat/kdd_cup_2023_DE/product_embeddings_2023-06-03-21-55-48.pt'
DE_test_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates_phase2/query_embeddings/NARM_Feat/kdd_cup_2023_DE/predict_embeddings_2023-06-03-21-56-34.pt'
JP_product_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates_phase2/query_embeddings/NARM_Feat/kdd_cup_2023_JP/product_embeddings_2023-06-03-21-57-16.pt'
JP_test_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates_phase2/query_embeddings/NARM_Feat/kdd_cup_2023_JP/predict_embeddings_2023-06-03-21-57-56.pt'
UK_product_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates_phase2/query_embeddings/NARM_Feat/kdd_cup_2023_UK/product_embeddings_2023-06-03-21-58-29.pt'
UK_test_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates_phase2/query_embeddings/NARM_Feat/kdd_cup_2023_UK/predict_embeddings_2023-06-03-21-59-05.pt'

In [9]:
@lru_cache(maxsize=1)
def read_merged_candidates_feature_test():
    return pd.read_parquet(merged_candidates_feature_test_path, engine='pyarrow')

@lru_cache(maxsize=1)
def read_test_sessions():
    return pd.read_csv(test_sessions_path)

In [10]:
merged_candidates = read_merged_candidates_feature_test()
test_sessions = read_test_sessions()
EMBED_DIM = 128
merged_candidates.sort_values(by=['sess_id', 'product'], inplace=True)
merged_candidates.reset_index(drop=True, inplace=True)

In [25]:
# sess embeddings 
test_DE_query_emb = torch.load(DE_test_embeddings_path, map_location='cpu')
test_JP_query_emb = torch.load(JP_test_embeddings_path, map_location='cpu')
test_UK_query_emb = torch.load(UK_test_embeddings_path, map_location='cpu')
test_query_embeddings = torch.cat(
    [test_DE_query_emb[test_sessions['locale'] == 'DE'], test_JP_query_emb[test_sessions['locale'] == 'JP'], test_UK_query_emb[test_sessions['locale'] == 'UK']],
    dim=0)

In [13]:
# product embeddings 
DE_product_emb = torch.load(DE_product_embeddings_path, map_location='cpu')
JP_product_emb = torch.load(JP_product_embeddings_path, map_location='cpu')
UK_product_emb = torch.load(UK_product_embeddings_path, map_location='cpu')
product_embeddings = torch.cat([DE_product_emb, JP_product_emb, UK_product_emb], dim=0)

In [14]:
DE_dataset_cache = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/b5aeac4e5b9ff0518bbcb59a28086594'
JP_dataset_cache = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/2536617955df215e0047f5b220d1c012'
UK_dataset_cache = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/8d133ea55ad67bd3efd625dfeff0fb1d'
DE_train_dataset, DE_valid_dataset = _load_cache(DE_dataset_cache)
JP_train_dataset, JP_valid_dataset = _load_cache(JP_dataset_cache)
UK_train_dataset, UK_valid_dataset = _load_cache(UK_dataset_cache)
locale_map = {
    'DE' : DE_train_dataset.field2token2idx['product_id'], 
    'JP' : JP_train_dataset.field2token2idx['product_id'], 
    'UK' : UK_train_dataset.field2token2idx['product_id']
    }

In [15]:
DE_product_list, DE_id_list = list(zip(*locale_map['DE'].items()))
JP_product_list, JP_id_list = list(zip(*locale_map['JP'].items()))
UK_product_list, UK_id_list = list(zip(*locale_map['UK'].items()))
product_list = list(DE_product_list) + list(JP_product_list) + list(UK_product_list)
id_list = list(DE_id_list) + list(JP_id_list) + list(UK_id_list)
locale_list = ['DE'] * len(DE_id_list) + ['JP'] * len(JP_id_list) + ['UK'] * len(UK_id_list)
product_id_df = pd.DataFrame({'locale' : locale_list, 'product' : product_list, 'dataset_id' : id_list})

In [16]:
merged_candidates_ = merged_candidates[['sess_id', 'sess_locale', 'product']].copy()

In [17]:
# merged_candidates_g = cudf.from_pandas(merged_candidates_)
# product_id_df_g = cudf.from_pandas(product_id_df)

In [18]:
# merged_candidates_score_g = merged_candidates_g.merge(product_id_df_g, how='left', left_on=['sess_locale', 'product'], right_on=['locale', 'product'])
# merged_candidates_score_g['dataset_id'] = merged_candidates_score_g['dataset_id'].fillna(0)
# merged_candidates_score_g.drop(columns=['locale'], inplace=True)
# merged_candidates_score_g = merged_candidates_score_g.sort_values(by=['sess_id', 'product'])
# merged_candidates_score_g.reset_index(drop=True, inplace=True)
# merged_candidates_score = merged_candidates_score_g.to_pandas()

In [19]:
merged_candidates_score = merged_candidates_.merge(product_id_df, how='left', left_on=['sess_locale', 'product'], right_on=['locale', 'product'])
merged_candidates_score['dataset_id'] = merged_candidates_score['dataset_id'].fillna(0)
merged_candidates_score.drop(columns=['locale'], inplace=True)
merged_candidates_score = merged_candidates_score.sort_values(by=['sess_id', 'product'])
merged_candidates_score.reset_index(drop=True, inplace=True)
assert len(merged_candidates_score) == len(merged_candidates)

In [20]:
# del merged_candidates_g
# del product_id_df_g
# del merged_candidates_score_g

In [21]:
locale_offset = {'DE' : 0, 'JP' : len(DE_product_list), 'UK' : len(DE_product_list) + len(JP_product_list)}
for locale in ['DE', 'JP', 'UK']:
    merged_candidates_score['dataset_id'][merged_candidates_score['sess_locale'] == locale] = \
        merged_candidates_score['dataset_id'][merged_candidates_score['sess_locale'] == locale] + locale_offset[locale]

/tmp/ipykernel_588299/2978421209.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates_score['dataset_id'][merged_candidates_score['sess_locale'] == locale] = \


In [26]:
test_query_embeddings = test_query_embeddings.to('cuda:0')
product_embeddings = product_embeddings.to('cuda:0')

In [27]:
merged_candidates_score[FIELD_NAME] = get_scores(merged_candidates_score, test_query_embeddings, product_embeddings)

  0%|          | 0/94294 [00:00<?, ?it/s]/tmp/ipykernel_588299/2438919930.py:10: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  batch_product_id = torch.tensor(batch_sess['dataset_id'].tolist(), dtype=torch.long, device=product_embeddings.device)
100%|██████████| 94294/94294 [03:00<00:00, 523.03it/s]


In [28]:
normalize_scores(merged_candidates_score, FIELD_NAME, 'normalized_'+FIELD_NAME)

In [29]:
merged_candidates[FIELD_NAME] = merged_candidates_score[FIELD_NAME]
merged_candidates['normalized_'+FIELD_NAME] = merged_candidates_score['normalized_'+FIELD_NAME]

In [60]:
cast_dtype(merged_candidates, [FIELD_NAME, 'normalized_'+FIELD_NAME])
merged_candidates.to_parquet(merged_candidates_feature_test_path, engine='pyarrow')

In [33]:
merged_candidates[(merged_candidates['sess_id'] == 250015)].sort_values(by='normalized_sasrec_scores_2', ascending=False)[
    ['sess_locale', 'product', 'normalized_sasrec_scores_2', 'sasrec_scores_2', 'normalized_narm_scores', 'narm_scores']
][:15]

,sess_locale,product,normalized_sasrec_scores_2,sasrec_scores_2,normalized_narm_scores,narm_scores
76100445,UK,B08PKLFM9T,0.076618,10.966581,0.046232,13.069794
76100587,UK,B0B1DD9QYC,0.054272,10.621758,0.030429,12.651532
76100437,UK,B08KRNJ14G,0.040197,10.321547,0.014734,11.926312
76100624,UK,B0B7MRLJH1,0.032691,10.114833,0.022032,12.328626
76100379,UK,B07L3CHH63,0.032596,10.111935,0.032247,12.709558
76100447,UK,B08PVH9GZG,0.027505,9.942128,0.006879,11.164602
76100518,UK,B09N75PFSD,0.022642,9.747556,0.035685,12.810847
76100543,UK,B09TPDVRT9,0.021482,9.694937,0.010047,11.543421
76100389,UK,B07RL1JQ5W,0.020788,9.662111,0.036073,12.821655
76100594,UK,B0B3GPL5JD,0.020658,9.655845,0.031890,12.698408
